In [1]:
import cv2 
import numpy as np
net = cv2.dnn.readNet('/Users/amariofausta/Documents/Kuliah/Semester 8/Bobot/yolov3custom_last.weights copy 5','/Users/amariofausta/Downloads/yolov3custom2.cfg')
with open("/Users/amariofausta/Documents/Kuliah/Semester 8/Dataset/dataset/classes.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]

In [2]:
classes

['Bola', 'Pembatas']

In [3]:
layer_names = net.getLayerNames()
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]

In [4]:
img = cv2.imread("/Users/amariofausta/Downloads/WhatsApp Image 2023-02-22 at 10.54.41 AM.jpeg")
height, width, channels = img.shape
blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False)


In [5]:
net.setInput(blob)
outs = net.forward(output_layers)

class_ids = []
confidences = []
boxes = []
for out in outs:
    for detection in out:
        scores = detection[5:]
        class_id = np.argmax(scores)
        confidence = scores[class_id]
        if confidence > 0.5:
            center_x = int(detection[0] * width)
            center_y = int(detection[1] * height)
            w = int(detection[2] * width)
            h = int(detection[3] * height)
            x = center_x - w / 2
            y = center_y - h / 2
            boxes.append([x, y, w, h])
            confidences.append(float(confidence))
            class_ids.append(class_id)

indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

font = cv2.FONT_HERSHEY_PLAIN
colors = np.random.uniform(0, 255, size=(len(classes), 3))

for i in range(len(boxes)):
    print(i,indexes)
    if i in indexes:
        x, y, w, h = boxes[i]
        label = str(classes[class_ids[i]])
        color = colors[class_ids[i]]
        cv2.rectangle(img, (int(x), int(y)), (int(x + w), int(y + h)), color, 2)
        cv2.putText(img, label, (int(x), int(y) - 5), font, 1, color, 2)

cv2.imshow("Image", img)
cv2.waitKey(0)
cv2.destroyAllWindows()

0 [0]
